# Libmark demo

This notebook demonstrates the usage of the Libmark package. 

## Pushing

Start by importing the relevant packages:

In [ ]:
import libmark as lm
import numpy as np
import tequila as tq

Checking available optimizers:

In [ ]:
print(lm.tracker.get_optimizers())

Define functions required later by the VQE loop:

In [ ]:
# Create a molecule (only H2 is properly supported, although other 2 atom molecules can also be used)
def create_H2(R, basis_set="sto-3g", transformation="Jordan-Wigner", **kwargs):
    geometry = f'H 0.0 0.0 0.0\nH 0.0 0.0 {R}'
    return tq.chemistry.Molecule(geometry=geometry, basis_set=basis_set, transformation=transformation, **kwargs)

def create_hamiltonian(molecule):
    return molecule.make_hamiltonian()

def make_uccsd_ansatz(molecule, trotter_steps, n_qubits=None, **kwargs): 
    return molecule.make_uccsd_ansatz(trotter_steps, **kwargs)

# Create molecules, in case of issues see psi4 error workaround in "Basic_VQE_example".
def create_molecules(molecule_function, distances):
    return [molecule_function(r) for r in distances]


Define distances and the optimizer to be used:

In [ ]:
optimizer = "Nelder-Mead"

step = 0.10
molecular_distances = np.arange(.0 + step, 2.0 + step, step)
molecules = create_molecules(create_H2, molecular_distances)

## The VQE loop

In [ ]:
def run_vqe(molecules, hamiltonian_function, ansatz_function, optimizer, **ansatz_kwargs):
    qresult = lm.tracker.get_tracker(optimizer, ' TOKEN HERE ') # Token can be aqcuired from the Quantmark website

    for i, molecule in enumerate(molecules):
        H = hamiltonian_function(molecule)
        U = ansatz_function(molecule=molecule, n_qubits=len(H.qubits), **ansatz_kwargs)
        E = tq.ExpectationValue(H=H, U=U)
        variables = {k:0.0 for k in U.extract_variables()}
        result = tq.minimize(objective=E, method=optimizer, initial_values=variables, silent=True)
        qresult.add_run(result, molecule, H, U) # Add the run
        print(f'Run {i+1}/{len(molecules)} completed')

    res = qresult.push() # Push results to the server
    print(res) # Prints the result id or a possible error message
    qresult.save() # Save data to a JSON file


Finally we can run the loop:

In [ ]:
ansatz_kwargs = {"trotter_steps": 5}
run_vqe(molecules, create_hamiltonian, make_uccsd_ansatz, optimizer, **ansatz_kwargs)

After running the "run_vqe" loop, the results should show up on the Quantmark website, in case of issues, make sure that the token is correct.

## Downloading

Results can be downloaded from the Quantmark website if you are the owner or if the result is public. In both cases, your personal token is needed.

A raw dump of all collected data can be downloaded like so:

In [ ]:
data = lm.api.get_data(' ID HERE ', ' TOKEN HERE ') # Returns a dictionary of all data available

While data from the above dictionary can be used to manually recreate the experiment, libmark also supports recalculating the molecular energies automatically:

In [ ]:
experiment = lm.api.get_experiment(' ID HERE ', ' TOKEN HERE ')
results = experiment.run_experiment() # Returns a list of tuples (distance, object returned by tq.minimize)

## Other methods

FCI energies for the H2 molecule can be fetched from the Quantmark website.
The supported basis_sets are "sto-3g", "6-31g" and "def2-QZVPPD"

In [ ]:
sto_3g_fci = lm.api.get_fci("sto-3g")
print(sto_3g_fci) # List of pairs (distance, energy)

The quantmark website has leaderboards that showcase the best performing algorithms. To include a result in these leaderboards, they need to calculate energies for predetermined distances. The currently accepted distances can be fetched from the api too:

In [15]:
benchmark_distances = lm.api.get_distances()
print(benchmark_distances) # List of distances

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]
